In [2]:
import sqlite3 
from sqlite3 import Error 

import pandas as pd

pd.set_option('display.max_columns', None)

## Условия

Дано 2 CSV-файла: один с транзакциями за период 01.01.2023 – 20.04.2023, второй с информацией о клиентах.

***За дефолтную дату для выборки действующих клиентов брать ‘2023-05-01’***

Необходимо в Jupyter- ноутбуке выполнить следующие пункты, используя SQLite:

# Шаг 1

Необходимо скачать CSV-файл - «[transactions_for_dz2.csv](https://drive.google.com/file/d/1GsKxPQ3cp1_uuyoLoAQLlsHTLEbXOMIo/view?usp=share_link)», создать таблицу transaction_bd со всеми полями, загрузить данные из файла в таблицу и оставить таблицу со структурой:

* (0, 'TRANSACTION_ID', 'INTEGER', 0, None, 0) - id транзакции
* (1, 'TX_DATETIME', 'NUMERIC', 0, None, 0) - дата транзакции
* (2, 'CUSTOMER_ID', 'INTEGER', 0, None, 0) - id клиента
* (3, 'TERMINAL_ID', 'INTEGER', 0, None, 0) - id терминала
* (4, 'TX_AMOUNT', 'REAL', 0, None, 0) - сумма транзакции

Также необходимо скачать второй CSV-файл - «[client_info.csv](https://drive.google.com/file/d/1oFzZfwHBtP0NOulBYlp4osIY5NIPIdn-/view?usp=share_link)», создать таблицу customer_bd со всеми полями, загрузить данные из файла в таблицу и получить таблицу со структурой:

* (0, 'CLIENT_ID', 'INTEGER', 0, None, 0) - id клиента
* (1, 'START_DT', 'NUMERIC', 0, None, 0) - дата начало записи о клиенте
* (2, 'END_DT', 'NUMERIC', 0, None, 0) - дата закрытия записи о клиенте
* (3, 'CLIENT_NAME', 'TEXT', 0, None, 0) - название клиента
* (4, 'YEAR_BIRTH', 'TEXT', 0, None, 0) - дата рождение клиента

**Примечание для следующих шагов:** сумма транзакций не может быть null-ом, это всегда число.

In [3]:
transactions = pd.read_csv('data/transactions_for_dz2.csv')
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   TRANSACTION_ID  1048575 non-null  int64  
 1   TX_DATETIME     1048575 non-null  object 
 2   CUSTOMER_ID     1048575 non-null  int64  
 3   TERMINAL_ID     1048575 non-null  int64  
 4   TX_AMOUNT       1048575 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 40.0+ MB


In [4]:
transactions.head(3)

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,0,2023-01-01 00:00:31,596,3156,533.07
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94


In [5]:
clients = pd.read_csv('data/client_info.csv', sep=';')
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4988 entries, 0 to 4987
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   START_DT     4988 non-null   object
 1   END_DT       4988 non-null   object
 2   CLIENT_NAME  4988 non-null   object
 3   YEAR_BIRTH   4988 non-null   int64 
 4   CLIENT_ID    4988 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 195.0+ KB


In [6]:
clients

,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH,CLIENT_ID
0,2015-07-16,2021-01-01,Olivia,1979,2213
1,2015-07-17,2021-01-01,Emma,1979,1148
2,2015-07-18,2021-01-01,Charlotte,1979,2293
3,2015-07-19,2021-01-01,Amelia,1979,1867
4,2015-07-20,2021-01-01,Ava,1979,1767
...,...,...,...,...,...
4983,2023-04-20,2999-12-31,Havilah,2000,4737
4984,2023-04-20,2999-12-31,Hazelyn,2000,2301
4985,2023-04-20,2999-12-31,Helaina,2000,3238
4986,2023-04-20,2999-12-31,Helene,2000,3946


In [7]:
def create_connection(path):

    """
    Функция подключения к базе данных
    """

    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [8]:
def execute_query(connection, query):

    """
    Функция выполнения запроса к базе данных
    """

    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [9]:
def sql_insert(connection, data, name, m=0, n=1):

    """
    Функция для добавления значений в таблицу
    """

    cursor = connection.cursor()
    try:
        cursor.executemany(f"""
                INSERT INTO {name} {tuple(data.columns[m:n])} 
                    VALUES ({str("?, " * (n-m-1) + "?")})""",
                data.iloc[:, m:n].values)
        connection.commit()
        print('Data inserted successfully')
    except Error as e:
        print(f"The error '{e}' occurred")

In [10]:
# Подключаюсь к базе данных (если нет такой, то создается)
connection = create_connection("HW_3.db")

Connection to SQLite DB successful


In [11]:
# Код для создания таблицы transaction_table
create_transaction_table = """
    CREATE TABLE IF NOT EXISTS transaction_table (
        transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
        tx_datetime NUMERIC NOT NULL,
        customer_id INTEGER NOT NULL,
        terminal_id INTEGER NOT NULL,
        tx_amount REAL NOT NULL
    );
    """

# Создание таблицы через функцию
execute_query(connection, create_transaction_table)

Query executed successfully


In [12]:
# Код для создания таблицы customer_table
create_customer_table = """
    CREATE TABLE IF NOT EXISTS customer_table (
        start_dt NUMERIC NOT NULL,
        end_dt NUMERIC NOT NULL,
        client_name TEXT NOT NULL,
        year_birth INTEGER NOT NULL,
        client_id INTEGER PRIMARY KEY
    );
    """

# Создание таблицы через функцию
execute_query(connection, create_customer_table)

Query executed successfully


In [13]:
# Вставка данных в таблицу transaction_table
sql_insert(connection, transactions, 'transaction_table', 0, transactions.shape[1])

The error 'UNIQUE constraint failed: transaction_table.transaction_id' occurred


In [14]:
# Вставка данных в таблицу transaction_table
sql_insert(connection, clients, 'customer_table', 0, clients.shape[1])

The error 'UNIQUE constraint failed: customer_table.client_id' occurred


In [15]:
def table_info(table_name, conn):
    c = conn.cursor()
    meta = c.execute("PRAGMA table_info('" + table_name + "')")
    for r in meta:
        print(r)

In [16]:
table_info('transaction_table', connection)

(0, 'transaction_id', 'INTEGER', 0, None, 1)
(1, 'tx_datetime', 'NUMERIC', 1, None, 0)
(2, 'customer_id', 'INTEGER', 1, None, 0)
(3, 'terminal_id', 'INTEGER', 1, None, 0)
(4, 'tx_amount', 'REAL', 1, None, 0)


In [17]:
table_info('customer_table', connection)

(0, 'start_dt', 'NUMERIC', 1, None, 0)
(1, 'end_dt', 'NUMERIC', 1, None, 0)
(2, 'client_name', 'TEXT', 1, None, 0)
(3, 'year_birth', 'INTEGER', 1, None, 0)
(4, 'client_id', 'INTEGER', 0, None, 1)


In [18]:
pd.read_sql('SELECT * FROM transaction_table', connection)

,transaction_id,tx_datetime,customer_id,terminal_id,tx_amount
0,0,2023-01-01 00:00:31,596,3156,533.07
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94
3,3,2023-01-01 00:09:29,4128,8737,620.65
4,4,2023-01-01 00:10:34,927,9906,490.66
...,...,...,...,...,...
1048570,1048570,2023-04-20 10:07:13,2380,3780,325.64
1048571,1048571,2023-04-20 10:07:28,738,5151,20.38
1048572,1048572,2023-04-20 10:07:33,1000,5417,182.79
1048573,1048573,2023-04-20 10:07:39,3028,6439,455.44


In [19]:
pd.read_sql('SELECT * FROM customer_table', connection)

,start_dt,end_dt,client_name,year_birth,client_id
0,2013-08-08,2999-12-31,Eleni,1972,0
1,2010-10-19,2999-12-31,Jocelyn,1954,1
2,2012-02-29,2999-12-31,Bria,1965,2
3,2017-01-09,2023-04-15,Analiah,1984,3
4,2011-05-06,2023-04-16,Kameron,1958,4
...,...,...,...,...,...
4983,2023-01-01,2999-12-31,Fatou,2000,4997
4984,2015-08-01,2021-01-01,Abigail,1979,4998
4985,2019-07-15,2023-04-15,Mulan,1996,4999
4986,2021-11-01,2999-12-31,Oksi,2000,11111


# Шаг 2

Написать скрипты:

In [20]:
default_date = '2023-05-01'

* a. Найти имя клиента/клиентов с максимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [21]:
pd.read_sql(f"""
    SELECT 
        tx.customer_id,
        ct.client_name, MAX(tx.tx_sum) AS max_sum, 
        date(ct.end_dt) as end_dt 
    FROM (
        SELECT
            customer_id, SUM(tx_amount) AS tx_sum
        FROM 
            transaction_table
        GROUP BY
            customer_id
    ) AS tx 
    LEFT JOIN customer_table ct
        ON tx.customer_id = ct.client_id
    WHERE date(ct.end_dt) >= date('{default_date}')
    """, connection)

,customer_id,client_name,max_sum,end_dt
0,2891,Kylee,786115.87,2999-12-31


* b. Найти имя клиента/клиентов с минимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [22]:
pd.read_sql(f"""
    SELECT 
        ct.client_name, MIN(tx.tx_sum) AS min_sum, 
        date(ct.end_dt) as end_dt 
    FROM (
        SELECT
            customer_id, SUM(tx_amount) AS tx_sum
        FROM 
            transaction_table
        GROUP BY
            customer_id
    ) AS tx 
    LEFT JOIN customer_table ct
        ON tx.customer_id = ct.client_id
    WHERE date(ct.end_dt) >= date('{default_date}')
    """, connection)

,client_name,min_sum,end_dt
0,Kloe,30.48,2999-12-31


* c. Найти сумму транзакций относительно даты 2023-04-01 для клиентов, у которых id начинается с 111 (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Транзакции учитываются только после завершении дня.

In [23]:
pd.read_sql(f"""
    SELECT
        date(tx.tx_datetime) AS tx_date, SUM(tx.tx_amount) AS tx_sum
    FROM 
        transaction_table tx
    INNER JOIN (
        SELECT
            client_id
        FROM customer_table
        WHERE client_id LIKE '111%'
            AND date(end_dt) >= date('{default_date}')
    ) AS ct
        ON tx.customer_id = ct.client_id
    WHERE date(tx_datetime) = '2023-04-01'
    """, connection)

,tx_date,tx_sum
0,2023-04-01,61211.55


* d. Найти сумму транзакций относительно года рождения клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию года рождения.

In [24]:
pd.read_sql(f"""
    SELECT
        ct.year_birth, SUM(tx.tx_amount) AS tx_sum
    FROM 
        transaction_table tx
    INNER JOIN (
        SELECT
            client_id, year_birth
        FROM customer_table
        WHERE date(end_dt) >= date('{default_date}')
    ) AS ct
        ON tx.customer_id = ct.client_id
    GROUP BY
        ct.year_birth
    ORDER BY
        ct.year_birth DESC
    """, connection)

,year_birth,tx_sum
0,2000,8.632720e+07
1,1999,1.041507e+06
2,1998,6.016750e+05
3,1997,8.479943e+06
4,1996,1.483253e+06
5,1995,6.321200e+06
6,1994,6.244281e+06
7,1993,5.732058e+06
8,1992,5.251433e+06
9,1991,7.141074e+06


* e. Найти количество транзакций для каждого клиента (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию количества транзакций.

In [25]:
pd.read_sql(f"""
    SELECT
        ct.client_id, ct.client_name, COUNT(tx.transaction_id) AS tx_count
    FROM 
        transaction_table tx
    INNER JOIN (
        SELECT
            client_id, client_name
        FROM customer_table
        WHERE date(end_dt) >= date('{default_date}')
    ) AS ct
        ON tx.customer_id = ct.client_id
    GROUP BY
        ct.client_id
    ORDER BY
        COUNT(tx.transaction_id) DESC
    """, connection)

,client_id,client_name,tx_count
0,2891,Kylee,477
1,2580,Cerenity,464
2,1953,Adara,455
3,2932,Michaela,454
4,3864,Claire,453
...,...,...,...
3731,1976,Becky,1
3732,1942,Perel,1
3733,1880,Roman,1
3734,1095,Marisa,1


# Шаг 3
Найти сумму транзакций за каждый месяц (январь, февраль, март, апрель) для всех клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [26]:
pd.read_sql(f"""
    SELECT
        strftime('%m', date(tx_datetime)) AS tx_month, SUM(tx.tx_amount) AS tx_sum
    FROM 
        transaction_table tx
    INNER JOIN (
        SELECT
            client_id
        FROM customer_table
        WHERE date(end_dt) >= date('{default_date}')
    ) AS ct
        ON tx.customer_id = ct.client_id
    GROUP BY
        strftime('%m', date(tx_datetime))
    """, connection)

,tx_month,tx_sum
0,01,1.159199e+08
1,02,1.062295e+08
2,03,1.150534e+08
3,04,7.261812e+07


In [27]:
connection.close()